# H3 Intro

In [3]:
import h3
import pandas as pd
import folium
import branca.colormap as cm
import xml.etree.ElementTree as ET
from datetime import datetime

#### Set Coordinates and Resolution

In [4]:

lat = 39.73632141579045
lng = -104.9897890191563
resolution = 10

#### Convert Coordinates to H3

In [8]:
h3_index = h3.latlng_to_cell(lat, lng, resolution)
print(f"H3 Index: {h3_index}")

H3 Index: 8a268cda80dffff


#### Convert H3 to Coordinates

In [9]:
lat_lng = h3.cell_to_latlng(h3_index)
print(f"Latitude/Longitude: {lat_lng}")


Latitude/Longitude: (39.73594029216176, -104.98910346554506)


#### Convert H3 to Geometry Boundary

In [10]:
boundary = h3.cell_to_boundary(h3_index)
print(f"Boundary: {boundary}")

Boundary: ((39.73524428010972, -104.98900125244737), (39.73564964864881, -104.98824237067122), (39.73634565833955, -104.98834457890632), (39.7366362996269, -104.98920568004662), (39.736230928869034, -104.9899645663659), (39.73553491904265, -104.98986234700183))


#### Map the Boundary

In [14]:
# Create a map
m = folium.Map(location=[lat, lng], zoom_start=16)

# Add the H3 cell to the map
folium.Polygon(locations=boundary, color='red', weight=2).add_to(m)

# Save the map
m


# Suunto gpx

In [1]:
# Fix for ET error: ensure ElementTree is imported
import xml.etree.ElementTree as ET

In [5]:
def parse_gpx_to_dataframe(gpx_file_path):
    """
    Parse a GPX file and convert it to a pandas DataFrame.
    Returns a DataFrame with columns: lat, lon, elevation, time, heart_rate
    """
    tree = ET.parse(gpx_file_path)
    root = tree.getroot()
    
    # Define namespaces
    namespaces = {
        'gpx': 'http://www.topografix.com/GPX/1/1',
        'gpxtpx': 'http://www.garmin.com/xmlschemas/TrackPointExtension/v1'
    }
    
    track_points = []
    
    # Find all track points
    for trkpt in root.findall('.//gpx:trkpt', namespaces):
        lat = float(trkpt.get('lat'))
        lon = float(trkpt.get('lon'))
        
        # Get elevation
        ele = trkpt.find('gpx:ele', namespaces)
        elevation = float(ele.text) if ele is not None else None
        
        # Get time
        time_elem = trkpt.find('gpx:time', namespaces)
        time_str = time_elem.text if time_elem is not None else None
        
        # Get heart rate from extensions
        hr_elem = trkpt.find('.//gpxtpx:hr', namespaces)
        heart_rate = int(hr_elem.text) if hr_elem is not None else None
        
        track_points.append({
            'lat': lat,
            'lon': lon,
            'elevation': elevation,
            'time': time_str,
            'heart_rate': heart_rate
        })
    
    df = pd.DataFrame(track_points)
    
    # Convert time to datetime
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'])
    
    return df

# Parse the GPX file
gpx_df = parse_gpx_to_dataframe('../data/test/suunto_train_example.gpx')
print(f"GPX data shape: {gpx_df.shape}")
print(f"Columns: {gpx_df.columns.tolist()}")
gpx_df.head()


GPX data shape: (2132, 5)
Columns: ['lat', 'lon', 'elevation', 'time', 'heart_rate']


,lat,lon,elevation,time,heart_rate
0,44.838780,-0.643135,27.8,2025-09-17 18:54:53+00:00,96
1,44.838772,-0.643102,27.8,2025-09-17 18:54:54+00:00,95
2,44.838762,-0.643058,27.8,2025-09-17 18:54:55+00:00,95
3,44.838747,-0.643015,27.8,2025-09-17 18:54:56+00:00,95
4,44.838737,-0.642967,27.8,2025-09-17 18:54:57+00:00,94


In [6]:
# Display basic statistics about the GPX data
print("GPX Data Summary:")
print(f"Total track points: {len(gpx_df)}")
print(f"Time range: {gpx_df['time'].min()} to {gpx_df['time'].max()}")
print(f"Duration: {gpx_df['time'].max() - gpx_df['time'].min()}")
print(f"Latitude range: {gpx_df['lat'].min():.6f} to {gpx_df['lat'].max():.6f}")
print(f"Longitude range: {gpx_df['lon'].min():.6f} to {gpx_df['lon'].max():.6f}")
print(f"Elevation range: {gpx_df['elevation'].min():.1f}m to {gpx_df['elevation'].max():.1f}m")
print(f"Heart rate range: {gpx_df['heart_rate'].min()} to {gpx_df['heart_rate'].max()} bpm")

# Check for missing values
print(f"\nMissing values:")
print(gpx_df.isnull().sum())


GPX Data Summary:
Total track points: 2132
Time range: 2025-09-17 18:54:53+00:00 to 2025-09-17 19:30:26+00:00
Duration: 0 days 00:35:33
Latitude range: 44.837590 to 44.840302
Longitude range: -0.643135 to -0.599462
Elevation range: 14.2m to 35.8m
Heart rate range: 94 to 181 bpm

Missing values:
lat           0
lon           0
elevation     0
time          0
heart_rate    0
dtype: int64


In [50]:
df_gpx = gpx_df[['lat', 'lon']].drop_duplicates()

df_gpx.to_csv('../data/gpx_treated_data.csv', index=False)

In [7]:
# Create a map showing the GPX track
# Calculate center point for the map
center_lat = gpx_df['lat'].mean()
center_lon = gpx_df['lon'].mean()

# Create the map
m = folium.Map(location=[center_lat, center_lon], tiles="Cartodb dark_matter", zoom_start=15)

# Add the track as a line
track_coords = list(zip(gpx_df['lat'], gpx_df['lon']))
folium.PolyLine(track_coords, color='red', weight=3, opacity=0.8).add_to(m)

# Add start and end markers
folium.Marker(
    location=[gpx_df['lat'].iloc[0], gpx_df['lon'].iloc[0]], 
    popup='Start', 
    icon=folium.Icon(color='green', icon='play')
).add_to(m)

folium.Marker(
    location=[gpx_df['lat'].iloc[-1], gpx_df['lon'].iloc[-1]], 
    popup='End', 
    icon=folium.Icon(color='red', icon='stop')
).add_to(m)

# Display the map
m


In [38]:
# Convert GPX track points to H3 indices
def add_h3_to_gpx_dataframe(df, resolution=10):
    """Add H3 index and boundaries to the GPX DataFrame"""
    h3_data = []
    for _, row in df.iterrows():
        lat = row['lat']
        lon = row['lon']
        h3_index = h3.latlng_to_cell(lat, lon, resolution)
        boundaries = h3.cell_to_boundary(h3_index)
        h3_data.append({
            'h3_index': h3_index,
            'h3_boundaries': boundaries
        })
    
    h3_df = pd.DataFrame(h3_data)
    return pd.concat([df, h3_df], axis=1)

# Add H3 data to the GPX DataFrame
gpx_df_with_h3 = add_h3_to_gpx_dataframe(gpx_df, resolution=9)

print("GPX DataFrame with H3 data:")
print(f"Shape: {gpx_df_with_h3.shape}")
print(f"Columns: {gpx_df_with_h3.columns.tolist()}")
print(f"Unique H3 cells: {gpx_df_with_h3['h3_index'].nunique()}")

# Show first few rows
gpx_df_with_h3[['lat', 'lon', 'elevation', 'heart_rate', 'h3_index']].head()


GPX DataFrame with H3 data:
Shape: (2132, 7)
Columns: ['lat', 'lon', 'elevation', 'time', 'heart_rate', 'h3_index', 'h3_boundaries']
Unique H3 cells: 19


,lat,lon,elevation,heart_rate,h3_index
0,44.838780,-0.643135,27.8,96,89186b6d86fffff
1,44.838772,-0.643102,27.8,95,89186b6d86fffff
2,44.838762,-0.643058,27.8,95,89186b6d86fffff
3,44.838747,-0.643015,27.8,95,89186b6d86fffff
4,44.838737,-0.642967,27.8,94,89186b6d86fffff


In [39]:
df_st_folium = gpx_df_with_h3[['h3_index', 'h3_boundaries']].drop_duplicates()

df_st_folium.to_csv('../data/app_csv_data.csv', index=False)

## Real Data: 311 Calls about illegal parking in Denver, Colorado

#### Import and filter data

In [ ]:
# Use gpx_df_with_h3 data
gpx_df_with_h3 = gpx_df_with_h3.dropna(subset=['lat', 'lon'])
gpx_df_with_h3.shape

(2132, 7)

In [59]:
gpx_df_with_h3 = gpx_df_with_h3[['h3_index']].drop_duplicates().reset_index(drop=True)
gpx_df_with_h3.shape

gpx_df_with_h3.to_csv('../data/gpx_treated_data.csv', index=False)

In [58]:
gpx_df_with_h3.head()

,h3_index
0,89186b6d86fffff
1,89186b6d867ffff
2,89186b6d863ffff
3,89186b6d82bffff
4,89186b6d823ffff


#### Visualize on a map

In [43]:
gpx_df_with_h3.head()

,h3_index,h3_boundaries
0,89186b6d86fffff,"((44.839922148484305, -0.6433260719029629), (4..."
87,89186b6d867ffff,"((44.840299894806726, -0.6391638536919598), (4..."
161,89186b6d863ffff,"((44.842837589023645, -0.6418529619431464), (4..."
239,89186b6d82bffff,"((44.84067747455567, -0.6350017506271951), (44..."
339,89186b6d823ffff,"((44.841054887744534, -0.630839762747802), (44..."


In [44]:

# Use gpx_df_with_h3 data to visualize H3 cells on a map


m = folium.Map(location=[center_lat, center_lon], zoom_start=15, tiles="Cartodb dark_matter")

# Add the H3 cells to the map
for _, row in gpx_df_with_h3.iterrows():
    print(row['h3_boundaries'])
    boundaries = row['h3_boundaries']
    # Convert tuple of tuples to list of [lat, lon] for folium
    locations = [[lat, lon] for lat, lon in boundaries]
    folium.Polygon(
        locations=locations,
        color='#EFBF04',
        weight=1,
        fill=True,
        fill_opacity=0.5,
        popup=f"H3 Index: {row['h3_index']}"
    ).add_to(m)

# Display the map
m


((44.839922148484305, -0.6433260719029629), (44.83882434727408, -0.6452045013732605), (44.83700660845641, -0.6447990734850095), (44.83628669409128, -0.6425153429869365), (44.83738447293523, -0.6406369880088075), (44.83920218850949, -0.6410422890367325))
((44.840299894806726, -0.6391638536919598), (44.83920218850949, -0.6410422890367325), (44.83738447293523, -0.6406369880088075), (44.83666448690879, -0.6383533784940029), (44.837762170846204, -0.6364750176553776), (44.83957986316881, -0.6368801918255591))
((44.842837589023645, -0.6418529619431464), (44.841739860363575, -0.6437314717912388), (44.839922148484305, -0.6433260719029629), (44.83920218850949, -0.6410422890367325), (44.840299894806726, -0.6391638536919598), (44.842117583440526, -0.6395691267102441))
((44.84067747455567, -0.6350017506271951), (44.83957986316881, -0.6368801918255591), (44.837762170846204, -0.6364750176553776), (44.83704211316928, -0.6341915291421264), (44.83813970220278, -0.6323131624638627), (44.83995737126546, -